In [9]:
from pprint import pprint
import pandas as pd
import mlflow
from mlflow.tracking import MlflowClient

backend_uri = os.environ.get('MLFLOW_TRACKING_URI')
mlflow.set_tracking_uri(backend_uri)
client = MlflowClient()

<IPython.core.display.Javascript object>

In [3]:
prediction_types = [
    { 'type': 'Regression', 'registered_as': 'linreg'},
    { 'type': 'Classification', 'registered_as': ''}
]

prediction_types = pd.DataFrame(prediction_types)
prediction_types.index += 1

In [4]:
prediction_types

,type,registered_as
1,Regression,linreg
2,Classification,


In [5]:
prediction_types[prediction_types['registered_as'] == '' ]

,type,registered_as
2,Classification,


In [18]:
for rm in client.list_registered_models():
    pprint(dict(dict(rm)['latest_versions'][0])['name'], indent=4)

'linreg'
'linreg-v2.0'


In [22]:
selected_ptype = 1
base_name = prediction_types.loc[selected_ptype]['registered_as']
names = []

for rm in client.list_registered_models():
    reg_name = dict(dict(rm)['latest_versions'][0])['name']
    if base_name in reg_name:
        names.append(reg_name)

In [26]:
registered_models = pd.DataFrame(columns=['source', 'version', 'stage', 'name'])
for name in names:
    for mv in client.search_model_versions(f"name='{name}'"):
        mv = dict(mv)
        data = {'source': mv['source'].split('/')[-1], 
                'version': mv['version'], 
                'stage': mv['current_stage'],
                'name': name
               }
        registered_models = registered_models.append(data, ignore_index = True)

registered_models.index += 1

In [27]:
registered_models

,source,version,stage,name
1,mlflow_tf_housing,1,Staging,linreg
2,mlflow_tf_housing,2,None,linreg
3,mlflow_custom_housing,3,None,linreg
4,mlflow_custom_housing,4,None,linreg
5,mlflow_sk_housing,1,None,linreg-v2.0
6,mlflow_sk_housing,2,Staging,linreg-v2.0


In [31]:
cond = (registered_models['stage'] == 'Staging')
for _, row in registered_models[cond].iterrows():
    print(f"{row['source']}:v{row['version']}")

mlflow_tf_housing:v1
mlflow_sk_housing:v2


In [49]:
name = "linreg-v2.0"
cond = '(stage == "Staging") and (name == @name)'
registered_models.query(cond)

,source,version,stage,name
6,mlflow_sk_housing,2,Staging,linreg-v2.0


In [32]:
df = registered_models[~cond]
df

,source,version,stage,name
2,mlflow_tf_housing,2,None,linreg
3,mlflow_custom_housing,3,None,linreg
4,mlflow_custom_housing,4,None,linreg
5,mlflow_sk_housing,1,None,linreg-v2.0


In [33]:
for index, row in registered_models[~cond].iterrows():
    print(f"{row['source']}:v{row['version']}")

mlflow_tf_housing:v2
mlflow_custom_housing:v3
mlflow_custom_housing:v4
mlflow_sk_housing:v1


In [27]:
current_version = int(registered_models[cond]['version'].iloc[0])
selected_version = 2

client.transition_model_version_stage(
    name = name,
    version = current_version,
    stage="Archived"
)

client.transition_model_version_stage(
    name = name,
    version = selected_version,
    stage="Staging"
)

<ModelVersion: creation_timestamp=1626929161471, current_stage='Staging', description=None, last_updated_timestamp=1626929571214, name='linreg', run_id='23320a23a02049ca935dbf22ecced8bb', run_link=None, source='s3://mlruns/23320a23a02049ca935dbf22ecced8bb/artifacts/mlflow_tf_housing', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [59]:
import subprocess
import json

out = subprocess.run(["kubectl", "describe", "svc", "mlflow-ligreg-infer"], 
                    stdout=subprocess.PIPE, encoding='utf-8')
print(out.stdout)

Name:                     mlflow-ligreg-infer
Namespace:                default
Labels:                   <none>
Annotations:              <none>
Selector:                 app=mlflow-ligreg-infer
Type:                     NodePort
IP:                       10.102.200.4
LoadBalancer Ingress:     localhost
Port:                     <unset>  1236/TCP
TargetPort:               1236/TCP
NodePort:                 <unset>  31236/TCP
Endpoints:                10.1.1.177:1236,10.1.1.178:1236
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>



In [68]:
cmd = "kubectl describe svc mlflow-ligreg-infer | findstr Endpoints"
ps = subprocess.Popen(cmd,shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT, encoding='utf-8')
output = ps.communicate()[0]
print(output.split(','))

['Endpoints:                10.1.1.177:1236', '10.1.1.178:1236\n']
